# Vorlesung 3: Erneuerbare und Speicheroptionen
## Übersicht Residuallast und Ausblick Speicherbedarf

In diesem Notebook werden Daten zur Stromerzeugung / -verbrauch in Deutschlangd analysiert (Quelle: Open Power System Data) 

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
# Plotting options
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Data
To explore this relation, we will use the time-series dataset from the Open Power System Data Based: [https://data.open-power-system-data.org/time_series](https://data.open-power-system-data.org/time_series)

It is curated from data published by ENTSO-E and includes load and renewable generation profiles from various regions within the European synchronized network. For now, we will focus on Germany in the year 2019.

In [ ]:
data = pd.read_csv("../data/time_series_60min_singleindex_filtered.csv", index_col=0, parse_dates=True)
data.index = data.index.tz_convert("Europe/Berlin") # convert timestap from UTC to local time

In [ ]:
# function to extract and pre-process the dataset for one specific region (a country or TSO).
def tso_dataframe(df, tso):
    wind_columns = [
        f"{tso}_wind_onshore_generation_actual",
        f"{tso}_wind_offshore_generation_actual",
    ]
    wind_columns = df.columns.intersection(wind_columns)   # some TSO regions have no offshore wind
    df_wind = df[wind_columns].sum(axis=1)                 # aggregate on- and off-shore wind generation
    df_wind = df_wind.rename("wind")

    df_solar = df[f"{tso}_solar_generation_actual"]
    df_solar = df_solar.rename("solar")

    df_load = df[f"{tso}_load_actual_entsoe_transparency"]
    df_load = df_load.rename("load")

    df = pd.concat([df_load, df_solar, df_wind], axis=1) # join load, solar and wind data
    return df

## Electricity demand and renewable generation in Germany.

The intermittent nature of renewable generation and the inflexible electricity consumption make it difficult to fit one into each other. Let's explore their temporal relationships. We can do so, by plotting the data in the form of time series, heatmaps and histograms.

In [ ]:
# read the 'Germany' dataset
df_DE = tso_dataframe(data, "DE")

In [ ]:
df_DE

In [ ]:
# function to plot load profile and aggregated renewable generation profiles
def plot_load_generation(df, **kwargs):
    fig = go.Figure()
    fig.update_layout(xaxis_title="Time", yaxis_title="Power [MW]", **kwargs)
    fig.add_trace(go.Scatter(x=df.index, y=df["load"],  name="load"))
    fig.add_trace(go.Scatter(x=df.index, y=df["solar"], name="solar", stackgroup="renewables"))
    fig.add_trace(go.Scatter(x=df.index, y=df["wind"],  name="wind",  stackgroup="renewables"))
    if "residual" in df.columns:
        fig.add_trace(go.Scatter(x=df.index, y=df["residual"],  name="residual"))

    return fig

In [ ]:
# function to plot heatmaps
def datetime_heatmap(df):
    fig = go.Figure(data=go.Heatmap(
        z=df,
        x=df.index.date,
        y=df.index.time,
        colorscale='RdBu_r'
    ))
    return fig

In [ ]:
plot_load_generation(df_DE.loc["2019-07"], template=template)

In [ ]:
datetime_heatmap(df_DE["load"]).update_layout(template=template, title="Power demand [MW] - Germany 2019")

In [ ]:
datetime_heatmap(df_DE["wind"]).update_layout(template=template, title="Wind power generation [MW]- Germany 2019")

In [ ]:
datetime_heatmap(df_DE["solar"]).update_layout(template=template, title="Solar power generation [MW] - Germany 2019")

In [ ]:
df_DE[["wind", "solar"]].plot.hist(template=template, log_y=True, histnorm="percent", labels={"value": "Power [MW]"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

## Residual load

Now that we have examined their individual features, let's see how load and renewable generation stand against each other. Our metric is the residual load, which is defined by subtracting the renewable generation to the power demand.

$$P_{res}(t) = P_{load}(t) - P_{solar}(t) - P_{wind}(t) \qquad \text{(F 1.7)}$$

From that we can derive the residual load duration curve, which may give us an insight into the relative magnitude and the utilization of the renewable generation.

In [ ]:
# function to add residual in a new column
def add_residual(df):
    df["residual"] = df["load"] - df["solar"] - df["wind"]
    return df

In [ ]:
df_DE = add_residual(df_DE)

In [ ]:
plot_load_generation(df_DE.loc["2019-07"], template=template)

In [ ]:
# function to plot the residual load duration curve for Germany
def plot_residual_curve_DE(df, **kwargs):
    fig = go.Figure()
    fig.update_layout(title="Residual load duration curve - 2019", xaxis_title="Time [h]", yaxis_title="Power [MW]", width=800, height=600, **kwargs)
    fig.add_trace(go.Scatter(x=np.arange(8760), y=np.zeros(8760), line={"color": "grey", "dash": "dash"}, opacity=0.7, showlegend=False))
    # fig.update_yaxes(exponentformat="power")
    aggregated_residual = df["residual"].sort_values(ascending=False).values
    time_hours = np.arange(aggregated_residual.size)
    fig.add_trace(go.Scatter(x=time_hours, y=aggregated_residual, name="Germany", line={"color": "#19d3f3"}))
    return fig

In [ ]:
plot_residual_curve_DE(df_DE, template=template)

## Electricty demand and renewable generation in Germany - Regional level

<img src="https://upload.wikimedia.org/wikipedia/commons/8/82/Regelzonen_mit_%C3%9Cbertragungsnetzbetreiber_in_Deutschland.png" align="left" width="400" alt="DE-TSOs-map">

On the figure left we can see the landscape of the Transmission system operators (TSOs) in Germany. The four TSOs in Germany are:
- Amprion
- Tennet
- Transnet BW
- 50hertz

Each TSO is responsible for providing the electric power transmission in their corresponding region and securing the reliability of the system by coordinating the operations and managing their power transmission infrastructure.

Disaggregating the data and can give us an insight on how the German power sector works in a regional level. Let's take a look and plot the residual load duration curve to compare.

In [ ]:
# create a dictionary with a dataframe to each TSO
df_tso = {tso: tso_dataframe(data, tso) for tso in ("DE_tennet", "DE_50hertz", "DE_amprion", "DE_transnetbw")}
df_tso["Germany"] = df_DE # also include the Germany-wide aggregated data

In [ ]:
# 50hertz's demand and generation in July 2019 as an example
plot_load_generation(df_tso["DE_50hertz"].loc["2019-07"], template=template)

In [ ]:
# calculate the residual to all TSOs
df_tso = {tso: add_residual(df) for tso, df in df_tso.items()}

In [ ]:
# function to plot the residual load duration curve for the TSOs
def plot_residual_curve(profiles, year="2019", **kwargs):
    fig = go.Figure()
    fig.update_layout(title="Residual load duration curve - "+year, xaxis_title="Time [h]", yaxis_title="Power [MW]", width=800, height=600, **kwargs)
    fig.add_trace(go.Scatter(x=np.arange(8760), y=np.zeros(8760), line={"color": "grey", "dash": "dash"}, opacity=0.7, showlegend=False, name=0))
    # fig.update_yaxes(exponentformat="power")
    for tso, df in profiles.items():
        aggregated_residual = df.loc[year, "residual"].sort_values(ascending=False).values
        time_hours = np.arange(aggregated_residual.size)
        fig.add_trace(go.Scatter(x=time_hours, y=aggregated_residual, name=tso))
    return fig

In [ ]:
plot_residual_curve(df_tso, template=template)